In [5]:
import os

from scipy import ndimage
import math
import json
import torch
import numpy as np
import pandas as pd

import cv2
import matplotlib

from matplotlib import pyplot as plt

from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv

from utils.yolo import get_teeth_ROI
from utils.edge import tooth_isolation
from utils.preprocess import get_labels_by_image, get_image_by_labels
from utils.preprocess import xyxy_reformat, rotate_bounding_boxes, rect_include_another

matplotlib.use('module://matplotlib_inline.backend_inline')
# %matplotlib inline

load_dotenv()


True

In [6]:
model = torch.hub.load(r'..\YOLO', 'custom', path=r'..\YOLO\weights\8-bound.pt', source='local')
data_dir = '..' / Path(os.getenv('DATASET_DIR'))


In [ ]:
target_labels = ['caries', 'filling', 'endo', 'post', 'crown', 'R.R']
# df = pd.read_csv('label_TF.csv', index_col='filename')
image_labels_df = get_image_by_labels()
# image_labels_df = df[target_labels]
# image_labels_df = get_image_by_labels(target_labels)[target_labels]

image_labels_df['crown'].sum()


In [ ]:
filename = image_labels_df.index[1]
filepath_image = data_dir / f'{filename}.jpg'
filepath_json = data_dir / f'{filename}.json'


In [ ]:
# Image
# Inference
results = model(filepath_image)

teeth_roi = get_teeth_ROI(results)
teeth_roi_images = teeth_roi['images'][filename]
teeth_roi_split_teeth = teeth_roi['split_teeth']
teeth_roi_images


In [ ]:
teeth_roi = get_teeth_ROI(results)
teeth_roi_images = teeth_roi['images'][filename]
teeth_roi_split_teeth = teeth_roi['split_teeth']
teeth_roi_images


# One file check

In [ ]:
image_labels = {}
labels = get_labels_by_image(filepath_json, target_labels)
for target_roi in teeth_roi_images:
    flag = target_roi['flag']
    target_roi_image = target_roi['image']
    tooth_position = target_roi['tooth_position']
    im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
    im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

    isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False)
    if not isolation_data:
        continue
    regions = isolation_data['crop_regions']
    theta = isolation_data['angle']
    offset = target_roi['offset']

    phi = math.radians(theta)
    for label in labels:
        xyxy = np.hstack(label['points'])  # [x, y, x, y]
        xyxy = xyxy_reformat(np.array([xyxy]))

        xyxy = xyxy - np.tile(offset, 2)
        if xyxy.min() < 0:
            continue

        xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
        xyxy = xyxy[0].astype(int)

        for tooth_number, region in regions.items():
            tooth_xyxy = region['xyxy']
            key = f'{filename} {tooth_number}'
            if not key in image_labels.keys():
                image_labels[key] = []
            if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                image_labels[key].append(label['label'])

image_labels


In [ ]:
import warnings

warnings.filterwarnings("ignore")

image_labels = {}
for filename in tqdm(image_labels_df.index):
    filepath_image = data_dir / f'{filename}.jpg'
    filepath_json = data_dir / f'{filename}.json'

    results = model(filepath_image)

    teeth_roi = get_teeth_ROI(results)
    teeth_roi_images = teeth_roi['images'][filename]
    teeth_roi_split_teeth = teeth_roi['split_teeth']

    labels = get_labels_by_image(filepath_json, target_labels)
    for target_roi in teeth_roi_images:
        target_roi_image = target_roi['image']
        flag = target_roi['flag']
        tooth_position = target_roi['tooth_position']

        im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
        im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

        isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False, save=True,
                                         filename=filename)
        if not isolation_data:
            continue
        regions = isolation_data['crop_regions']
        theta = isolation_data['angle']
        offset = target_roi['offset']

        phi = math.radians(theta)
        for label in labels:
            xyxy = np.hstack(label['points'])  # [x, y, x, y]
            xyxy = xyxy_reformat(np.array([xyxy]))

            xyxy = xyxy - np.tile(offset, 2)
            if xyxy.min() < 0:
                continue

            xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
            xyxy = xyxy[0].astype(int)

            for tooth_number, region in regions.items():
                tooth_xyxy = region['xyxy']
                key = f'{filename} {tooth_number}'
                if not key in image_labels.keys():
                    image_labels[key] = []
                if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                    image_labels[key].append(label['label'])


In [ ]:
target_roi


In [ ]:
# for tooth_number, r in region.items():
#     print(r)
# target_roi
# label
# !jupyter nbconvert --to script prepare_classify_format.ipynb

j = json.dumps(image_labels)

with open('image_labels_for_classify.json', 'w') as f:
    f.write(j)


## Check crop and origin image annotation count.

In [ ]:
crop_annotation_number = {}
for k, v in image_labels.items():
    filename, tooth_number = k.split(' ')

    if filename not in crop_annotation_number.keys():
        crop_annotation_number[filename] = {i: 0 for i in target_labels}

    for i in v:
        crop_annotation_number[filename][i] += 1

counter_diff = {}
for filename, counter in crop_annotation_number.items():
    counter_diff[filename] = {}

    org_number_row = image_labels_df.loc[filename]
    for k in target_labels:
        counter_diff[filename][k] = org_number_row.loc[k] - counter[k]

counter_diff_df = pd.DataFrame.from_dict(counter_diff, orient='index')
counter_diff_df.loc[(counter_diff_df > 2).any(axis=1)]

